In [1]:
from __future__ import print_function

import cv2
import PIL
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import np_utils
#from keras.regularizers import l2, activity_l2
from keras.regularizers import l2, l1
from keras.layers import deserialize as layer_from_config
import numpy as np
import _pickle as cPickle
import numpy
import cv2
import scipy
import csv
import dataprocessing

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def model_generate():
    img_rows, img_cols = 48, 48

    model = Sequential()
    model.add(Convolution2D(64, 5, 5, border_mode='valid', input_shape=(img_rows, img_cols, 1)))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(2, 2), dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(5, 5),strides=(2, 2)))

    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
    model.add(Convolution2D(64, 3, 3))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th')) 
    model.add(Convolution2D(64, 3, 3))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))

    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
    model.add(Convolution2D(128, 3, 3))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
    model.add(Convolution2D(128, 3, 3))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
 
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
    model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))
 
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(Dropout(0.2))
    model.add(Dense(1024))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(Dropout(0.2))

    model.add(Dense(7))
    model.add(Activation('softmax'))

    ada = Adadelta(lr=0.1, rho=0.95, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy',
    optimizer=ada,
    metrics=['accuracy'])
    model.summary()
    return model

In [3]:
def ConvertTo3DVolume(data):
    img_rows, img_cols = 48, 48
    test_set_x = numpy.asarray(data) 
    test_set_x = test_set_x.reshape(test_set_x.shape[0],img_rows,img_cols)
    test_set_x = test_set_x.reshape(test_set_x.shape[0], 1, img_rows, img_cols)
    test_set_x = test_set_x.astype('float32')
    return test_set_x

In [4]:
def predict_prob(number,test_set_x,model):
    toreturn = []
    for data5 in test_set_x:
        if number ==0:
            toreturn.append(dataprocessing.Flip(data5))
        elif number ==1:
            toreturn.append(dataprocessing.Roated15Left(data5))
        elif number ==2:
            toreturn.append(dataprocessing.Roated15Right(data5))
        elif number ==3:
            toreturn.append(dataprocessing.shiftedUp20(data5))
        elif number ==4:
            toreturn.append(dataprocessing.shiftedDown20(data5))
        elif number ==5:
            toreturn.append(dataprocessing.shiftedLeft20(data5))
        elif number ==6:
            toreturn.append(dataprocessing.shiftedRight20(data5))
        elif number ==7:
            toreturn.append(data5)
        toreturn = ConvertTo3DVolume(toreturn)
        proba = model.predict_proba(toreturn)
        return proba

In [5]:
model = model_generate()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(48, 48, 1..., padding="valid")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(2, 2), data_format="channels_first")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # This is added back by Interactiv

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 44, 44, 64)        1664      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 44, 44, 64)        123904    
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 44, 48, 68)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 22, 68)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 20, 24, 70)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 22, 64)        40384     
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 18, 22, 64)        25344     
__________

In [49]:
#filepath='MPM5.hdf5'
#model.load_weights(filepath)

In [6]:
X_train, y_train, X_test, y_test  = dataprocessing.load_data()

In [7]:
from numpy import array

In [8]:
for i in range(len(X_train)):
    X_train[i] = (array(X_train[i])).reshape(48, 48, 1)

In [9]:
type(X_train[0])


numpy.ndarray

In [10]:
X_train= array(X_train)

In [11]:
X_train.shape

(32181, 48, 48, 1)

In [12]:
for i in range(len(X_test)):
    X_test[i] = (array(X_test[i])).reshape(48, 48, 1)

In [13]:
X_test= array(X_test)

In [14]:
X_test.shape

(3589, 48, 48, 1)

In [15]:
y_train = array(y_train)
y_test = array(y_test)

In [16]:
print(y_train.shape, y_test.shape)

(32181,) (3589,)


In [17]:
# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, 7)
Y_test = np_utils.to_categorical(y_test, 7)

In [18]:
Y_train[:10]


array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

In [19]:
#Callback for saving the best model
from keras.callbacks import ModelCheckpoint
filepath= "FaceExp_Reco_tr_01.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [20]:
batch_size = 128
nb_classes = 7
epochs = 1200
img_channels = 1

In [21]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,    # randomly flip images
        fill_mode='nearest')

In [22]:
'''model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(X_train, Y_train,
          batch_size=batch_size),
          epochs=300,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=callbacks_list)

Epoch 1/300
252/252 [==============================] - 117s 464ms/step - loss: 1.8300 - acc: 0.2473 - val_loss: 1.8131 - val_acc: 0.2449

Epoch 00001: val_acc improved from -inf to 0.24492, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 2/300
252/252 [==============================] - 21s 85ms/step - loss: 1.8112 - acc: 0.2517 - val_loss: 1.8115 - val_acc: 0.2449

Epoch 00002: val_acc did not improve from 0.24492
Epoch 3/300
252/252 [==============================] - 21s 85ms/step - loss: 1.8079 - acc: 0.2517 - val_loss: 1.8061 - val_acc: 0.2499

Epoch 00003: val_acc improved from 0.24492 to 0.24993, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 4/300
252/252 [==============================] - 21s 85ms/step - loss: 1.8048 - acc: 0.2542 - val_loss: 1.7959 - val_acc: 0.2508

Epoch 00004: val_acc improved from 0.24993 to 0.25077, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 5/300
252/252 [==============================] - 22s 86ms/step - loss: 1.7994 - acc: 0.2586 - val_los


Epoch 00040: val_acc did not improve from 0.40596
Epoch 41/300
252/252 [==============================] - 22s 85ms/step - loss: 1.6047 - acc: 0.3712 - val_loss: 1.4809 - val_acc: 0.4146

Epoch 00041: val_acc improved from 0.40596 to 0.41460, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 42/300
252/252 [==============================] - 21s 85ms/step - loss: 1.5984 - acc: 0.3725 - val_loss: 1.5461 - val_acc: 0.3873

Epoch 00042: val_acc did not improve from 0.41460
Epoch 43/300
252/252 [==============================] - 22s 85ms/step - loss: 1.5898 - acc: 0.3794 - val_loss: 1.5136 - val_acc: 0.4057

Epoch 00043: val_acc did not improve from 0.41460
Epoch 44/300
252/252 [==============================] - 22s 86ms/step - loss: 1.5803 - acc: 0.3826 - val_loss: 1.4971 - val_acc: 0.4118

Epoch 00044: val_acc did not improve from 0.41460
Epoch 45/300
252/252 [==============================] - 21s 85ms/step - loss: 1.5806 - acc: 0.3822 - val_loss: 1.4876 - val_acc: 0.4154

Epoch 00045: v

Epoch 82/300
252/252 [==============================] - 22s 85ms/step - loss: 1.4429 - acc: 0.4473 - val_loss: 1.3760 - val_acc: 0.4675

Epoch 00082: val_acc did not improve from 0.48175
Epoch 83/300
252/252 [==============================] - 22s 85ms/step - loss: 1.4364 - acc: 0.4490 - val_loss: 1.3292 - val_acc: 0.4870

Epoch 00083: val_acc improved from 0.48175 to 0.48704, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 84/300
252/252 [==============================] - 21s 85ms/step - loss: 1.4349 - acc: 0.4485 - val_loss: 1.3851 - val_acc: 0.4622

Epoch 00084: val_acc did not improve from 0.48704
Epoch 85/300
252/252 [==============================] - 22s 86ms/step - loss: 1.4311 - acc: 0.4520 - val_loss: 1.3963 - val_acc: 0.4636

Epoch 00085: val_acc did not improve from 0.48704
Epoch 86/300
252/252 [==============================] - 22s 85ms/step - loss: 1.4300 - acc: 0.4521 - val_loss: 1.3635 - val_acc: 0.4784

Epoch 00086: val_acc did not improve from 0.48704
Epoch 87/300
25

252/252 [==============================] - 22s 86ms/step - loss: 1.3392 - acc: 0.4892 - val_loss: 1.2467 - val_acc: 0.5233

Epoch 00123: val_acc did not improve from 0.52828
Epoch 124/300
252/252 [==============================] - 22s 86ms/step - loss: 1.3374 - acc: 0.4890 - val_loss: 1.2365 - val_acc: 0.5302

Epoch 00124: val_acc improved from 0.52828 to 0.53023, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 125/300
252/252 [==============================] - 22s 86ms/step - loss: 1.3379 - acc: 0.4904 - val_loss: 1.2336 - val_acc: 0.5261

Epoch 00125: val_acc did not improve from 0.53023
Epoch 126/300
252/252 [==============================] - 22s 85ms/step - loss: 1.3326 - acc: 0.4889 - val_loss: 1.2573 - val_acc: 0.5222

Epoch 00126: val_acc did not improve from 0.53023
Epoch 127/300
252/252 [==============================] - 22s 86ms/step - loss: 1.3271 - acc: 0.4949 - val_loss: 1.2434 - val_acc: 0.5183

Epoch 00127: val_acc did not improve from 0.53023
Epoch 128/300
252/252 [=

252/252 [==============================] - 22s 86ms/step - loss: 1.2625 - acc: 0.5198 - val_loss: 1.2069 - val_acc: 0.5403

Epoch 00164: val_acc did not improve from 0.56032
Epoch 165/300
252/252 [==============================] - 22s 86ms/step - loss: 1.2628 - acc: 0.5182 - val_loss: 1.1807 - val_acc: 0.5456

Epoch 00165: val_acc did not improve from 0.56032
Epoch 166/300
252/252 [==============================] - 22s 85ms/step - loss: 1.2600 - acc: 0.5216 - val_loss: 1.1746 - val_acc: 0.5520

Epoch 00166: val_acc did not improve from 0.56032
Epoch 167/300
252/252 [==============================] - 21s 85ms/step - loss: 1.2539 - acc: 0.5237 - val_loss: 1.1907 - val_acc: 0.5425

Epoch 00167: val_acc did not improve from 0.56032
Epoch 168/300
252/252 [==============================] - 21s 85ms/step - loss: 1.2511 - acc: 0.5260 - val_loss: 1.1797 - val_acc: 0.5447

Epoch 00168: val_acc did not improve from 0.56032
Epoch 169/300
252/252 [==============================] - 22s 86ms/step - l

252/252 [==============================] - 21s 85ms/step - loss: 1.1991 - acc: 0.5484 - val_loss: 1.1392 - val_acc: 0.5634

Epoch 00207: val_acc did not improve from 0.57063
Epoch 208/300
252/252 [==============================] - 21s 85ms/step - loss: 1.1991 - acc: 0.5466 - val_loss: 1.1325 - val_acc: 0.5645

Epoch 00208: val_acc did not improve from 0.57063
Epoch 209/300
252/252 [==============================] - 22s 85ms/step - loss: 1.1940 - acc: 0.5468 - val_loss: 1.1215 - val_acc: 0.5692

Epoch 00209: val_acc did not improve from 0.57063
Epoch 210/300
252/252 [==============================] - 22s 85ms/step - loss: 1.1933 - acc: 0.5448 - val_loss: 1.1160 - val_acc: 0.5765

Epoch 00210: val_acc improved from 0.57063 to 0.57648, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 211/300
252/252 [==============================] - 22s 85ms/step - loss: 1.1917 - acc: 0.5467 - val_loss: 1.1292 - val_acc: 0.5656

Epoch 00211: val_acc did not improve from 0.57648
Epoch 212/300
252/252 [=

252/252 [==============================] - 22s 85ms/step - loss: 1.1548 - acc: 0.5642 - val_loss: 1.0886 - val_acc: 0.5926

Epoch 00249: val_acc improved from 0.58651 to 0.59264, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 250/300
252/252 [==============================] - 22s 85ms/step - loss: 1.1537 - acc: 0.5634 - val_loss: 1.1204 - val_acc: 0.5726

Epoch 00250: val_acc did not improve from 0.59264
Epoch 251/300
252/252 [==============================] - 21s 85ms/step - loss: 1.1440 - acc: 0.5651 - val_loss: 1.1184 - val_acc: 0.5843

Epoch 00251: val_acc did not improve from 0.59264
Epoch 252/300
252/252 [==============================] - 22s 85ms/step - loss: 1.1474 - acc: 0.5653 - val_loss: 1.0753 - val_acc: 0.5848

Epoch 00252: val_acc did not improve from 0.59264
Epoch 253/300
252/252 [==============================] - 21s 85ms/step - loss: 1.1484 - acc: 0.5632 - val_loss: 1.1110 - val_acc: 0.5756

Epoch 00253: val_acc did not improve from 0.59264
Epoch 254/300
252/252 [=


Epoch 00291: val_acc did not improve from 0.59766
Epoch 292/300
252/252 [==============================] - 21s 85ms/step - loss: 1.1101 - acc: 0.5791 - val_loss: 1.0696 - val_acc: 0.5954

Epoch 00292: val_acc did not improve from 0.59766
Epoch 293/300
252/252 [==============================] - 22s 85ms/step - loss: 1.1140 - acc: 0.5815 - val_loss: 1.0617 - val_acc: 0.5913

Epoch 00293: val_acc did not improve from 0.59766
Epoch 294/300
252/252 [==============================] - 22s 86ms/step - loss: 1.1099 - acc: 0.5802 - val_loss: 1.0556 - val_acc: 0.5991

Epoch 00294: val_acc improved from 0.59766 to 0.59905, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 295/300
252/252 [==============================] - 22s 85ms/step - loss: 1.1103 - acc: 0.5833 - val_loss: 1.0638 - val_acc: 0.5957

Epoch 00295: val_acc did not improve from 0.59905
Epoch 296/300
252/252 [==============================] - 22s 86ms/step - loss: 1.1053 - acc: 0.5806 - val_loss: 1.0804 - val_acc: 0.5899

Epoch 002

In [ ]:

model.fit_generator(datagen.flow(X_train, Y_train,
          batch_size=batch_size),
          epochs=600,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=callbacks_list)

Epoch 1/600
252/252 [==============================] - 21s 85ms/step - loss: 1.1031 - acc: 0.5824 - val_loss: 1.0888 - val_acc: 0.5893

Epoch 00001: val_acc did not improve from 0.60045
Epoch 2/600
252/252 [==============================] - 21s 84ms/step - loss: 1.1066 - acc: 0.5815 - val_loss: 1.0584 - val_acc: 0.5968

Epoch 00002: val_acc did not improve from 0.60045
Epoch 3/600
252/252 [==============================] - 21s 84ms/step - loss: 1.1017 - acc: 0.5807 - val_loss: 1.0846 - val_acc: 0.5879

Epoch 00003: val_acc did not improve from 0.60045
Epoch 4/600
252/252 [==============================] - 21s 85ms/step - loss: 1.0994 - acc: 0.5863 - val_loss: 1.0690 - val_acc: 0.5862

Epoch 00004: val_acc did not improve from 0.60045
Epoch 5/600
252/252 [==============================] - 21s 85ms/step - loss: 1.0981 - acc: 0.5833 - val_loss: 1.0552 - val_acc: 0.5940

Epoch 00005: val_acc did not improve from 0.60045
Epoch 6/600
252/252 [==============================] - 22s 86ms/step -

252/252 [==============================] - 21s 85ms/step - loss: 1.0745 - acc: 0.5929 - val_loss: 1.0513 - val_acc: 0.5893

Epoch 00044: val_acc did not improve from 0.60685
Epoch 45/600
252/252 [==============================] - 21s 85ms/step - loss: 1.0703 - acc: 0.5967 - val_loss: 1.0499 - val_acc: 0.5963

Epoch 00045: val_acc did not improve from 0.60685
Epoch 46/600
252/252 [==============================] - 21s 85ms/step - loss: 1.0711 - acc: 0.5949 - val_loss: 1.0494 - val_acc: 0.6004

Epoch 00046: val_acc did not improve from 0.60685
Epoch 47/600
252/252 [==============================] - 21s 85ms/step - loss: 1.0696 - acc: 0.5981 - val_loss: 1.0330 - val_acc: 0.6024

Epoch 00047: val_acc did not improve from 0.60685
Epoch 48/600
252/252 [==============================] - 21s 85ms/step - loss: 1.0670 - acc: 0.5960 - val_loss: 1.0273 - val_acc: 0.6088

Epoch 00048: val_acc improved from 0.60685 to 0.60880, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 49/600
252/252 [======

252/252 [==============================] - 24s 93ms/step - loss: 1.0439 - acc: 0.6064 - val_loss: 1.0401 - val_acc: 0.6032

Epoch 00087: val_acc did not improve from 0.61103
Epoch 88/600
252/252 [==============================] - 22s 87ms/step - loss: 1.0435 - acc: 0.6062 - val_loss: 1.0398 - val_acc: 0.6027

Epoch 00088: val_acc did not improve from 0.61103
Epoch 89/600
252/252 [==============================] - 22s 87ms/step - loss: 1.0430 - acc: 0.6077 - val_loss: 1.0362 - val_acc: 0.6066

Epoch 00089: val_acc did not improve from 0.61103
Epoch 90/600
252/252 [==============================] - 22s 87ms/step - loss: 1.0385 - acc: 0.6105 - val_loss: 1.0522 - val_acc: 0.5988

Epoch 00090: val_acc did not improve from 0.61103
Epoch 91/600
252/252 [==============================] - 22s 87ms/step - loss: 1.0410 - acc: 0.6109 - val_loss: 1.0493 - val_acc: 0.5979

Epoch 00091: val_acc did not improve from 0.61103
Epoch 92/600
252/252 [==============================] - 22s 86ms/step - loss: 

252/252 [==============================] - 22s 86ms/step - loss: 1.0190 - acc: 0.6143 - val_loss: 1.0514 - val_acc: 0.5999

Epoch 00130: val_acc did not improve from 0.61410
Epoch 131/600
252/252 [==============================] - 22s 88ms/step - loss: 1.0197 - acc: 0.6191 - val_loss: 1.0245 - val_acc: 0.6160

Epoch 00131: val_acc improved from 0.61410 to 0.61605, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 132/600
252/252 [==============================] - 22s 87ms/step - loss: 1.0170 - acc: 0.6144 - val_loss: 1.0273 - val_acc: 0.6127

Epoch 00132: val_acc did not improve from 0.61605
Epoch 133/600
252/252 [==============================] - 21s 85ms/step - loss: 1.0163 - acc: 0.6153 - val_loss: 1.0704 - val_acc: 0.5963

Epoch 00133: val_acc did not improve from 0.61605
Epoch 134/600
252/252 [==============================] - 22s 86ms/step - loss: 1.0107 - acc: 0.6183 - val_loss: 1.0411 - val_acc: 0.6060

Epoch 00134: val_acc did not improve from 0.61605
Epoch 135/600
252/252 [=

252/252 [==============================] - 23s 90ms/step - loss: 0.9925 - acc: 0.6238 - val_loss: 1.0272 - val_acc: 0.6099

Epoch 00173: val_acc did not improve from 0.61884
Epoch 174/600
252/252 [==============================] - 22s 88ms/step - loss: 1.0001 - acc: 0.6230 - val_loss: 1.0627 - val_acc: 0.6055

Epoch 00174: val_acc did not improve from 0.61884
Epoch 175/600
252/252 [==============================] - 22s 86ms/step - loss: 0.9927 - acc: 0.6271 - val_loss: 1.0381 - val_acc: 0.6069

Epoch 00175: val_acc did not improve from 0.61884
Epoch 176/600
252/252 [==============================] - 23s 91ms/step - loss: 0.9950 - acc: 0.6241 - val_loss: 1.0264 - val_acc: 0.6088

Epoch 00176: val_acc did not improve from 0.61884
Epoch 177/600
252/252 [==============================] - 23s 90ms/step - loss: 0.9958 - acc: 0.6288 - val_loss: 1.0261 - val_acc: 0.6119

Epoch 00177: val_acc did not improve from 0.61884
Epoch 178/600
252/252 [==============================] - 22s 89ms/step - l

...

252/252 [==============================] - 22s 86ms/step - loss: 0.9349 - acc: 0.6500 - val_loss: 1.0428 - val_acc: 0.6119

Epoch 00303: val_acc did not improve from 0.62998
Epoch 304/600
252/252 [==============================] - 22s 89ms/step - loss: 0.9413 - acc: 0.6446 - val_loss: 1.0085 - val_acc: 0.6227

Epoch 00304: val_acc did not improve from 0.62998
Epoch 305/600
252/252 [==============================] - 22s 86ms/step - loss: 0.9383 - acc: 0.6483 - val_loss: 1.0269 - val_acc: 0.6202

Epoch 00305: val_acc did not improve from 0.62998
Epoch 306/600
252/252 [==============================] - 22s 88ms/step - loss: 0.9304 - acc: 0.6523 - val_loss: 1.0260 - val_acc: 0.6149

Epoch 00306: val_acc did not improve from 0.62998
Epoch 307/600
252/252 [==============================] - 22s 87ms/step - loss: 0.9366 - acc: 0.6474 - val_loss: 1.0379 - val_acc: 0.6183

Epoch 00307: val_acc did not improve from 0.62998
Epoch 308/600
252/252 [==============================] - 21s 84ms/step - l

252/252 [==============================] - 22s 87ms/step - loss: 0.9178 - acc: 0.6564 - val_loss: 1.0255 - val_acc: 0.6211

Epoch 00346: val_acc did not improve from 0.63137
Epoch 347/600
252/252 [==============================] - 22s 88ms/step - loss: 0.9191 - acc: 0.6556 - val_loss: 1.0328 - val_acc: 0.6202

Epoch 00347: val_acc did not improve from 0.63137
Epoch 348/600
252/252 [==============================] - 22s 88ms/step - loss: 0.9159 - acc: 0.6565 - val_loss: 1.0082 - val_acc: 0.6283

Epoch 00348: val_acc did not improve from 0.63137
Epoch 349/600
252/252 [==============================] - 23s 90ms/step - loss: 0.9185 - acc: 0.6570 - val_loss: 1.0438 - val_acc: 0.6197

Epoch 00349: val_acc did not improve from 0.63137
Epoch 350/600
252/252 [==============================] - 23s 89ms/step - loss: 0.9187 - acc: 0.6551 - val_loss: 1.0358 - val_acc: 0.6172

Epoch 00350: val_acc did not improve from 0.63137
Epoch 351/600
252/252 [==============================] - 22s 87ms/step - l


Epoch 00389: val_acc did not improve from 0.63137
Epoch 390/600
252/252 [==============================] - 22s 88ms/step - loss: 0.9026 - acc: 0.6626 - val_loss: 1.0163 - val_acc: 0.6319

Epoch 00390: val_acc improved from 0.63137 to 0.63193, saving model to FaceExp_Reco_tr_01.best.hdf5
Epoch 391/600
252/252 [==============================] - 22s 87ms/step - loss: 0.9033 - acc: 0.6604 - val_loss: 1.0364 - val_acc: 0.6197

Epoch 00391: val_acc did not improve from 0.63193
Epoch 392/600
252/252 [==============================] - 22s 87ms/step - loss: 0.8973 - acc: 0.6634 - val_loss: 1.0336 - val_acc: 0.6219

Epoch 00392: val_acc did not improve from 0.63193
Epoch 393/600
252/252 [==============================] - 22s 87ms/step - loss: 0.9029 - acc: 0.6606 - val_loss: 1.0260 - val_acc: 0.6152

Epoch 00393: val_acc did not improve from 0.63193
Epoch 394/600
252/252 [==============================] - 22s 87ms/step - loss: 0.8968 - acc: 0.6648 - val_loss: 1.0108 - val_acc: 0.6264

Epoch 003

252/252 [==============================] - 23s 90ms/step - loss: 0.8910 - acc: 0.6680 - val_loss: 1.0190 - val_acc: 0.6239

Epoch 00433: val_acc did not improve from 0.63193
Epoch 434/600
252/252 [==============================] - 22s 87ms/step - loss: 0.8908 - acc: 0.6692 - val_loss: 1.0397 - val_acc: 0.6191

Epoch 00434: val_acc did not improve from 0.63193
Epoch 435/600
252/252 [==============================] - 22s 86ms/step - loss: 0.8893 - acc: 0.6674 - val_loss: 1.0307 - val_acc: 0.6213

Epoch 00435: val_acc did not improve from 0.63193
Epoch 436/600
252/252 [==============================] - 22s 88ms/step - loss: 0.8864 - acc: 0.6668 - val_loss: 1.0246 - val_acc: 0.6286

Epoch 00436: val_acc did not improve from 0.63193
Epoch 437/600
252/252 [==============================] - 21s 85ms/step - loss: 0.8864 - acc: 0.6698 - val_loss: 1.0101 - val_acc: 0.6308

Epoch 00437: val_acc did not improve from 0.63193
Epoch 438/600
252/252 [==============================] - 21s 85ms/step - l

252/252 [==============================] - 22s 88ms/step - loss: 0.8680 - acc: 0.6754 - val_loss: 1.0277 - val_acc: 0.6328

Epoch 00476: val_acc did not improve from 0.63360
Epoch 477/600
252/252 [==============================] - 22s 88ms/step - loss: 0.8778 - acc: 0.6730 - val_loss: 1.0186 - val_acc: 0.6322

Epoch 00477: val_acc did not improve from 0.63360
Epoch 478/600
252/252 [==============================] - 23s 90ms/step - loss: 0.8758 - acc: 0.6710 - val_loss: 1.0421 - val_acc: 0.6255

Epoch 00478: val_acc did not improve from 0.63360
Epoch 479/600
252/252 [==============================] - 22s 88ms/step - loss: 0.8706 - acc: 0.6731 - val_loss: 1.0349 - val_acc: 0.6230

Epoch 00479: val_acc did not improve from 0.63360
Epoch 480/600
 69/252 [=======>......................] - ETA: 15s - loss: 0.8688 - acc: 0.6749